A number of things were tested for this model

* A variety of different parameters to countVectorizer and tfidfVectorizor including
  * a number of different preprocessing steps
    * regular expressions to remove some oddities
    * an attempt to correct spellings and conjoined words
  * ngrams from (1,10) - code not here as it did not have any success
* Use of GridSearchCV to find the optimal alpha for each classifier
  
Interesting observations

* Spell checking can be very intensive and so care has to be given to it.  The inital attempt
was very slow and needed to be optimized.  It went from 10 minutes per 1000 messages (in our
data set) to 3.5 minutes with some rearrangment.  Even with that, the total time on my laptop
for a single use of it was 8 hours for transforming dev and training sets.
* There was no single good model or set of parameters that worked best for for all labels in the development
set.  The choice of scoring metric would determine which was best.  When we look at AUC Bernoulli Naive 
Bayes scores best, however Multinomial give the highest F1 scores.
* Preprocessing was ineffective.  We suspect though that this may simply have been the choice of preprocessing
style and that further effort may have yielded gains.

Results:  
Best scores for AUC:

<table>
<tr><th> label </th><th> model </th><th> alpha </th><th> type </th><th> preprocessor </th><th> tokenizer </th><th> max_features </th><th> stop_words </th><th> lowercase </th><th> strip_accents </th><th> aucdev</th></tr>
<tr><td> toxic </td><td> bern </td><td> 0.5 </td><td> count </td><td> 0 </td><td> 0 </td><td>  </td><td> english </td><td> TRUE </td><td> unicode </td><td> 0.852321727</td></tr>
<tr><td> severe_toxic </td><td> bern </td><td> 2.0 </td><td> count </td><td> 0 </td><td> 0 </td><td> 4000 </td><td> english </td><td> TRUE </td><td> ascii </td><td> 0.942104059</td></tr>
<tr><td> obscene </td><td> bern </td><td> 10.0 </td><td> count </td><td> 0 </td><td> 0 </td><td> 4000 </td><td> english </td><td> TRUE </td><td> ascii </td><td> 0.893736805</td></tr>
<tr><td> threat </td><td> bern </td><td> 0.5 </td><td> count </td><td> 0 </td><td> 0 </td><td> 5000 </td><td> english </td><td> TRUE </td><td>  </td><td> 0.838735331</td></tr>
<tr><td> insult </td><td> bern </td><td> 10.0 </td><td> count </td><td> 0 </td><td> 0 </td><td> 4000 </td><td> english </td><td> TRUE </td><td> ascii </td><td> 0.878101445</td></tr>
<tr><td> identity_hate </td><td> bern </td><td> 2.0 </td><td> count </td><td> 0 </td><td> 0 </td><td> 6000 </td><td>  </td><td> TRUE </td><td> ascii </td><td> 0.82812169</td></tr>
<tr><td> </td></tr>
<tr><td> Average </td><td> &nbsp; </td><td> &nbsp;</td><td> &nbsp; </td><td> &nbsp; </td><td> &nbsp; </td><td> &nbsp; </td><td>  </td><td> &nbsp; </td><td> &nbsp; </td><td> 0.872187</td></tr>
<tr><td> </td></tr>
</table>

Best scores for F1:
<table>
<tr><th> label </th><th> model </th><th> alpha </th><th> type </th><th> preprocessor </th><th> tokenizer </th><th> max_features </th><th> stop_words </th><th> lowercase </th><th> strip_accents </th><th> f1dev </th></tr>
<tr><td> toxic </td><td> multi </td><td> 0.1 </td><td> tfidf </td><td> 0 </td><td> 0 </td><td> 6000 </td><td> english </td><td> TRUE </td><td> ascii </td><td> 0.950819672 </td></tr>
<tr><td> severe_toxic </td><td> multi </td><td> 0.5 </td><td> tfidf </td><td> 0 </td><td> 0 </td><td> 10000 </td><td> english </td><td> TRUE </td><td> ascii </td><td> 0.990763086 </td></tr>
<tr><td> obscene </td><td> multi </td><td> 0.5 </td><td> tfidf </td><td> 0 </td><td> 0 </td><td> 4000 </td><td> english </td><td> TRUE </td><td> ascii </td><td> 0.9735999 </td></tr>
<tr><td> threat </td><td> multi </td><td> 0.1 </td><td> tfidf </td><td> 0 </td><td> 0 </td><td> 4000 </td><td> english </td><td> TRUE </td><td>  </td><td> 0.996879421 </td></tr>
<tr><td> insult </td><td> multi </td><td> 0.1 </td><td> tfidf </td><td> 0 </td><td> 0 </td><td> 5000 </td><td>  </td><td> TRUE </td><td> ascii </td><td> 0.969231089 </td></tr>
<tr><td> identity_hate </td><td> multi </td><td> 0.5 </td><td> tfidf </td><td> 0 </td><td> 0 </td><td> 6000 </td><td> english </td><td> TRUE </td><td>
 ascii </td><td> 0.991303986 </td></tr>
<tr><td> Average </td><td> &nbsp; </td><td> &nbsp; </td><td> &nbsp; </td><td> &nbsp; </td><td> &nbsp; </td><td> &nbsp; </td><td> &nbsp; </td><td> &nbsp; </td><td> &nbsp; </td><td> 0.978766 </td></tr>  
</table>

In [1]:
import numpy as np
import pandas as pd

#sklearn imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer

#scipy imports
from scipy.sparse import hstack

#Visualization imports
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 
import bokeh
#! pip install bokeh

# target classes
target_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
# read frames localy through csv
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")

# Random index generator for splitting training data
# Note: Each rerun of cell will create new splits.
randIndexCut = np.random.rand(len(train_df)) < 0.7

#S plit up data
test_data = test_df["comment_text"]
dev_data, dev_labels = train_df[~randIndexCut]["comment_text"], train_df[~randIndexCut][target_names]
train_data, train_labels = train_df[randIndexCut]["comment_text"], train_df[randIndexCut][target_names]

print('total training observations:', train_df.shape[0])
print('training data shape:', train_data.shape)
print('training label shape:', train_labels.shape)
print('dev label shape:', dev_labels.shape)
print('labels names:', target_names)

total training observations: 159571
training data shape: (111503,)
training label shape: (111503, 6)
dev label shape: (48068, 6)
labels names: ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


In [4]:
# Imports etc. used in this analysis
import string
import datetime
import re
from collections import Counter

from enchant import DictWithPWL
from enchant.checker import SpellChecker
import difflib

from sklearn import metrics

punctuation = "[\!\?\"\#\$\%\&\(\)\*\+\,\.\/\:\;\<\=\>\?\@\[\]\^\_\`\{\|\}\~\']"

In [5]:
# from http://norvig.com/spell-correct.html
# This is the Norvig spell checker and requires the storage of a "big.txt"
# file with a corpus of words that it uses for predictions

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('../data/big.txt').read()))

def norvig_P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def norvig_correction(word): 
    "Most probable spelling correction for word."
    return max(norvig_candidates(word), key=norvig_P)

def norvig_candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [7]:
# My own spell checking and corrections, using a combination of Norvig
# and pyenchant for spelling and a customer word splitter that uses
# both to verify words

## Custom Dictionary for pyenchant in ./data/mywords.txt.  There are many missing words
# mywords.txt currently contains:
# - list of firstnames and surnames gathered from several internet searches
#         http://www.birkenhoerdt.net/surnames-all.php?tree=1
# - List of swear words from: https://www.noswearing.com/dictionary
# - Custom entries that were flagged as misspelled but are known

my_dict=DictWithPWL('en_US', "../data/mywords.txt")
my_checker = SpellChecker(my_dict)


def trysplit(word):
    """This function looks for between 2 and 4 words which have been conjoined
    likethisforexample.  It uses pyenchant to recognize the words and then the
    probabilities assigned by Norvig, and returns the highest combined probability
    for the parsed block with all valid words
    
    Note: it also only accepts 'a' and 'i' as legitimate single letter words.  Various
    dictionaries define all individual letters as nouns, however they are rarely used
    in writing and if they are in conjoined words it will make them too difficult to
    process.
    
    Also no spelling corrections are attempted here, if the words are both misspelled
    and conjoined we give up:-)
    
    Args:
        word (string) : A word that is suspected to be conjoined
    Returns:
        list of strings : A list of up to 4 valid subwords
    """
    split_candidates = []
    max_proba = 0.0
    for i in range(1,len(word)):
        # I will only allow single letters of 'a' and 'i', all others ignored.  Pyenchant allows for
        # any single letter to be a legitimate word, and so too does norvig.  The dictionary defines
        # them as nouns that represent the letter, however even though several can be used in slang
        # (e.g. k->okay, c->see, u->you) using them in conjoined words would make the splitting far
        # too difficult and also human understanding much more difficult #howucthisk, u c?
        if (len(word[:i]) != 1 or (word[:i].lower() == 'a' or word[:i].lower() == 'i')) and (
            len(word[i:]) != 1 or (word[i:].lower() == 'a' or word[i:].lower() == 'i')):
            if my_checker.check(word[:i]) and my_checker.check(word[i:]):
                norvig_score = norvig_P(word[:i]) + norvig_P(word[i:])
                if norvig_score > max_proba:
                    max_proba = norvig_score
                    split_candidates = [word[:i],word[i:]]
                    
    for i in range(1,len(word)):
        for j in range(i+1,len(word)):        
            if (len(word[:i]) != 1 or (word[:i].lower() == 'a' or word[:i].lower() == 'i')) and (
                len(word[i:j]) != 1 or (word[i:j].lower() == 'a' or word[i:j].lower() == 'i')) and (
                len(word[i:]) != 1 or (word[i:].lower() == 'a' or word[i:].lower() == 'i')):
                
                if my_checker.check(word[:i]) and my_checker.check(word[i:j]) and my_checker.check(word[j:]):
                    norvig_score = norvig_P(word[:i]) + norvig_P(word[i:j]) + norvig_P(word[j:])
                    if norvig_score > max_proba:
                        max_proba = norvig_score
                        split_candidates = [word[:i],word[i:j],word[j:]]
                        
    for i in range(1,len(word)):
        for j in range(i+1,len(word)):
            for k in range(j+1,len(word)):
                if (len(word[:i]) != 1 or (word[:i].lower() == 'a' or word[:i].lower() == 'i')) and (
                    len(word[i:j]) != 1 or (word[i:j].lower() == 'a' or word[i:j].lower() == 'i')) and (
                    len(word[j:k]) != 1 or (word[j:k].lower() == 'a' or word[j:k].lower() == 'i')) and (
                    len(word[k:]) != 1 or (word[k:].lower() == 'a' or word[k:].lower() == 'i')):
                    if my_checker.check(word[:i]) and my_checker.check(word[i:j]) and my_checker.check(word[j:k]) and my_checker.check(word[k:]):
                        norvig_score = norvig_P(word[:i]) + norvig_P(word[i:j]) + norvig_P(word[j:k]) + norvig_P(word[k:])
                        if norvig_score > max_proba:
                            max_proba = norvig_score
                            split_candidates = [word[:i],word[i:j],word[j:k],word[k:]]
                            
    return split_candidates


def get_best_candidates(word):
    """ This function returns the highest probability candidate(s) for a
    word using pyenchant
    
    Args:
        word (string): single word that needs to be corrected
    Returns:
        list of equal probabilty spelling corrections
    """
    best_words = []
    best_ratio = 0
    a = set(my_checker.suggest(word))
    for b in a:
        if not '-' in b:
            tmp = difflib.SequenceMatcher(None, word, b).ratio()
            if tmp > best_ratio:
                best_words=[b]
                best_ratio=tmp
            elif tmp == best_ratio:
                best_words.append(b)
    return best_words

    
def fix_spellings(textinput):
    """This function takes the input text, parses it and then checks all words to correct
    any misspellings or conjoined words
    
    Args:
        block of text (string): a message to be split and checked for errors
    Returns:
        List of the split and corrected words
    """
    words = textinput.split()
    return_list = []
    for word in words:
        if my_checker.check(word) or my_checker.check(word.lower()) or word in punctuation or\
            any(i.isdigit() for i in word) or (word[-1].lower() == 's' and my_checker.check(word[:-1].lower())):
            return_list.append(word)
            # continue
        else:            
            candidates = get_best_candidates(word)
            if len(candidates) == 1:
                return_list.append(candidates.pop())
            elif len(candidates) > 1:
                # try another spell checker
                nv_candidates = norvig_candidates(word)
                tmp_set = set(nv_candidates).intersection(set(candidates))
                if len(tmp_set) == 1:
                    # only 1 overlap, should be correct
                    return_list.append(tmp_set.pop())
                elif len(nv_candidates) == 1 and next(iter(nv_candidates)) == word:
                        # this is suspicious, pyenchants' "suggest" method always returns something, however if
                        # norvigs method cannot find a suitable match within a short distance then it simply
                        # returns the orignal word.  This section is for potentially conjoined words
                        tmp_list=trysplit(word)

                        # If we get back a list of split words then use these
                        if len(tmp_list) != 0:
                            return_list.extend(tmp_list)
                            continue
                else:
                    # arbitrary now, just going to use the first one found from pyenchant, even though
                    # I have seen norvig get the correct word sometimes when pyenchant gets it wrong
                    return_list.append(candidates[0])
    return return_list

In [10]:
# Preprocessing functions:

def my_preprocessor_eng(textblock):
    """ This function is a simple set of regular expressions to remove/replace some punctuation
    and replace some abbreviations
    
    Args:
        textbloc (string): a string of words to run the expressions against
    Returns:
        a string of adjusted text
    """
    return_words = textblock
    return_words = re.sub(r"[^A-Za-z0-9]?!'`:´()", " ", return_words)
    return_words = re.sub(r","," ",return_words)
    return_words = re.sub(r"\.\.+"," ",return_words)
    return_words = re.sub(r"\."," ",return_words)
    return_words = re.sub(r"\("," ", return_words)
    return_words = re.sub(r"\)"," ", return_words)
    return_works = re.sub(r"\;", " ", return_words)
    return_words = re.sub(r":"," ", return_words)
    return_words = re.sub(r"´", "'", return_words)
    return_words = re.sub(r"`", "'", return_words)
    return_words = re.sub(r"''+", "'", return_words)
    return_words = re.sub(r" '", " ", return_words)
    return_words = re.sub(r"' ", " ", return_words)
    return_words = re.sub(r"\"", " ", return_words)
    return_words = re.sub(r"\/", " ", return_words)
    return_words = re.sub(r"\!\!+", "!!", return_words)
    return_words = re.sub(r"\?\?+", "?!", return_words)
    return_words = re.sub(r"\!", " !", return_words)
    return_words = re.sub(r"\?", " ?", return_words)
    return_words = re.sub(r"\b\d+\b", "999", return_words)
    # slang and abbreviations, need to be aware of capitolization and spaces
    return_words = re.sub(r"[Ww]on't", "will not", return_words)
    return_words = re.sub(r"n't", " not", return_words)
    return_words = re.sub(r"'s\b", " is", return_words)
    return_words = re.sub(r"\b[Aa]bt\b", "about", return_words)
    return return_words

In [11]:
# Calculation of scores on dev set and training set
def score_f1_auc_on_train_dev(dev_vector, train_vector, name, ctype='multi'):
    """This function creates a Naive Bayes classifier with the input vectors
    and then calculates both the AUC score and F1 score for the training and dev data.
    
    Args:
        dev_vector: the processed vector of dev data
        train_vector: the processed vector of training data
        name (string) : the label name to test
        ctype: multi, gaus or bern, choses between multinomial or bernoulli
    Returns:
        alpha: the best alpha value for this classifier
        f1scoredev: the F1 score for dev
        aucdev: the AUC score for dev
        f1scoretrain: the F1 score for training
        auctrain: the AUC score for training
    """
    alphas = {'alpha': [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]}

    if ctype == 'multi':
        nb_class = MultinomialNB().fit(train_vector, train_labels[name])
    elif ctype == 'bern':
        nb_class = BernoulliNB().fit(train_vector, train_labels[name])
    elif ctype == 'gaus':
        nb_class = GaussianNB().fit(train_vector, train_labels[name])
    else:
        print('ctype = %s, error' % (ctype))
    
    # use this to generate the best fitting model
    clf = GridSearchCV(nb_class, param_grid = alphas)
    clf.fit(train_vector, train_labels[name])
    
    predicted_labels_dev = clf.predict(dev_vector)
    fpr, tpr, thresholds = metrics.roc_curve(dev_labels[name], predicted_labels_dev)
    
    predicted_labels_train = clf.predict(train_vector)
    fpr1, tpr1, thresholds1 = metrics.roc_curve(train_labels[name], predicted_labels_train)
    
    f1scoredev = metrics.f1_score(dev_labels[name],predicted_labels_dev,average='micro')
    f1scoretrain = metrics.f1_score(train_labels[name],predicted_labels_train,average='micro')
    
    aucdev = metrics.auc(fpr,tpr)
    auctrain = metrics.auc(fpr1,tpr1)
    
    return clf.best_params_, f1scoredev,aucdev,f1scoretrain,auctrain

In [12]:
vectors_all=pd.DataFrame(columns=['vectortrain', 'vectordata','type','preprocessor', 'tokenizer',
                                  'max_features', 'stop_words', 'lowercase', 'strip_accents' ])

# this set of loops works through the chosen parameters creating a count and tfidf vectorizer
# and using the preprocessor or not (4 per iteration).  These are stored in the vectors_all
# dataframe along with a list of the parameters that were used to create each on
print(str(datetime.datetime.now().time()))
for i in None, 1000, 4000, 5000, 6000, 10000:
    print('%s: Doing i = %s' %(str(datetime.datetime.now().time()), i))
    for x in None, 'english':
        for y in None, 'ascii', 'unicode':
            for z in True, False:
                vect = CountVectorizer(max_features=i, stop_words=x, strip_accents=y, lowercase=z)
                vect_train = vect.fit_transform(train_data)
                vect_dev = vect.transform(dev_data)
                vectors_all.loc[vectors_all.shape[0]] = [vect_train, vect_dev, 'count', 0, 0, i, x, z, y]
                # Same but with the preprocessor
                vect = CountVectorizer(max_features=i, stop_words=x, 
                                strip_accents=y, lowercase=z, preprocessor=my_preprocessor_eng)
                vect_train = vect.fit_transform(train_data)
                vect_dev = vect.transform(dev_data)
                vectors_all.loc[vectors_all.shape[0]] = [vect_train, vect_dev, 'count', 1, 0, i, x, z, y]
                vect = TfidfVectorizer(max_features=i, stop_words=x, strip_accents=y, lowercase=z)
                vect_train = vect.fit_transform(train_data)
                vect_dev = vect.transform(dev_data)
                vectors_all.loc[vectors_all.shape[0]] = [vect_train, vect_dev, 'tfidf', 0, 0, i, x, z, y]
                # Same but with the preprocessor
                vect = TfidfVectorizer(max_features=i, stop_words=x, 
                                strip_accents=y, lowercase=z, preprocessor=my_preprocessor_eng)
                vect_train = vect.fit_transform(train_data)
                vect_dev = vect.transform(dev_data)
                vectors_all.loc[vectors_all.shape[0]] = [vect_train, vect_dev, 'tfidf', 1, 0, i, x, z, y]

print(str(datetime.datetime.now().time()))
print(vectors_all.shape)

18:56:07.267736
18:56:07.268452: Doing i = None
19:11:24.643492: Doing i = 1000
19:24:12.875430: Doing i = 4000
19:42:47.614669: Doing i = 5000
20:09:50.301146: Doing i = 6000
20:45:42.239538: Doing i = 10000
21:48:07.579829
(288, 9)


In [ ]:
def calculate_f1auc_all_models (vectors_all):
    """This function takes a vector of type vectors_all (defined above) acts as a wrapper
    to send each vector to the score_f1_auc_on_train_dev function selecting first multinomialNB
    and after that Bernoulli.  It collects the resulting scores and the best alpha and stores
    the results in a dataframe which is returned once all the results are calculated
    
    Args:
        Vectors_all (datafram) : a datafram defined above that stores the vector data in each row
    Returns
        dataframe: A dataframe of all the resulting scores and the details for each model
    """
    data_all=pd.DataFrame(columns=['vectorno', 'label', 'model','alpha', 'type', 'preprocessor', 'tokenizer', 
                                   'max_features', 'stop_words', 'lowercase', 'strip_accents',
                                   'f1dev','aucdev','f1train','auctrain'])
    for index,row in vectors_all.iterrows():
        for name in target_names:
            alpha, tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain = score_f1_auc_on_train_dev(
                train_vector=row['vectortrain'],
                dev_vector=row[1],name=name, ctype='multi')
            data_all.loc[data_all.shape[0]] = [index,name,'multi',alpha,row['type'], 
                                row['preprocessor'], row['tokenizer'], row['max_features'], 
                                row['stop_words'], row['lowercase'], row['strip_accents'],
                                tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain]
            alpha, tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain = score_f1_auc_on_train_dev(
                train_vector=row[0],
                dev_vector=row[1],name=name, ctype='bern')
            data_all.loc[data_all.shape[0]] = [index,name,'bern',alpha,row['type'], 
                                row['preprocessor'], row['tokenizer'], row['max_features'], 
                                row['stop_words'], row['lowercase'], row['strip_accents'],
                                tmpf1dev,tmpaucdev,tmpf1train,tmpauctrain]
    print('done')
    return data_all
    

In [112]:
# separated out.  This took over 24 hours to run as the fix_spellings tokenizer
# seems to have been very slow.  As an alternative we will preprocess our data
# before generating the dataframe

print(str(datetime.datetime.now().time()))
count_vect_plain_pre_token10k = CountVectorizer(tokenizer=fix_spellings, max_features=10000, 
                                                strip_accents='ascii', lowercase=True)
print(str(datetime.datetime.now().time()))
X_train_counts_plain_pre_token10k = count_vect_plain_pre_token6k.fit_transform(train_data)
print(str(datetime.datetime.now().time()))
X_dev_counts_plain_pre_token10k = count_vect_plain_pre_token6k.transform(dev_data)
print(str(datetime.datetime.now().time()))

In [ ]:
# insert the vectors from the previous cell into the large dataframe of vectors
vectors_all.loc[vectors_all.shape[0]] = [X_train_counts_plain_pre_token6k, 
                                         X_dev_counts_plain_pre_token6k,
                                         'count', 0, 1, 10000, None, True, 'ascii']

In [17]:
# Finally pull everything together and look for the top results for
# both F1 and AUC scores

top_aucf1_results = pd.DataFrame(columns=['vectorno', 'label', 'model','alpha', 'type', 'preprocessor', 'tokenizer', 
                                   'max_features', 'stop_words', 'lowercase', 'strip_accents',
                                   'f1dev','aucdev','f1train','auctrain'])

# calculate the f1 and auc for all the models
resultdf = calculate_f1auc_all_models(vectors_all)

# extract the top F1 and AUC scores from the calculations
for label in target_names:
    df_tmp = resultdf.loc[resultdf['label'] == label]
    top_aucf1_results.loc[top_aucf1_results.shape[0]] = df_tmp.loc[df_tmp['aucdev'].idxmax()]
    top_aucf1_results.loc[top_aucf1_results.shape[0]] = df_tmp.loc[df_tmp['f1dev'].idxmax()]

# Print the top AUC and F1 Scores
print(top_aucf1_results)

done


NameError: name 'top_auc_results' is not defined

In [22]:
# Print the top AUC and F1 Scores
print(top_aucf1_results)

    vectorno          label  model            alpha   type  preprocessor  \
0         40          toxic   bern   {'alpha': 0.5}  count             0   
1        226          toxic  multi   {'alpha': 0.1}  tfidf             0   
2        128   severe_toxic   bern   {'alpha': 2.0}  count             0   
3        274   severe_toxic  multi   {'alpha': 0.5}  tfidf             0   
4        128        obscene   bern  {'alpha': 10.0}  count             0   
5        130        obscene  multi   {'alpha': 0.5}  tfidf             0   
6        168         threat   bern   {'alpha': 0.5}  count             0   
7        122         threat  multi   {'alpha': 0.1}  tfidf             0   
8        128         insult   bern  {'alpha': 10.0}  count             0   
9        154         insult  multi   {'alpha': 0.1}  tfidf             0   
10       200  identity_hate   bern   {'alpha': 2.0}  count             0   
11       226  identity_hate  multi   {'alpha': 0.5}  tfidf             0   

    tokeniz

In [24]:
pd.DataFrame(top_aucf1_results).to_csv('f1auc_scores.csv')

In [21]:
top_aucf1_results = pd.DataFrame(columns=['vectorno', 'label', 'model','alpha', 'type', 'preprocessor', 'tokenizer', 
                                   'max_features', 'stop_words', 'lowercase', 'strip_accents',
                                   'f1dev','aucdev','f1train','auctrain'])

for label in target_names:
    df_tmp = resultdf.loc[resultdf['label'] == label]
    top_aucf1_results.loc[top_aucf1_results.shape[0]] = df_tmp.loc[df_tmp['aucdev'].idxmax()]
    top_aucf1_results.loc[top_aucf1_results.shape[0]] = df_tmp.loc[df_tmp['f1dev'].idxmax()]
    
print(top_aucf1_results)

    vectorno          label  model            alpha   type  preprocessor  \
0         40          toxic   bern   {'alpha': 0.5}  count             0   
1        226          toxic  multi   {'alpha': 0.1}  tfidf             0   
2        128   severe_toxic   bern   {'alpha': 2.0}  count             0   
3        274   severe_toxic  multi   {'alpha': 0.5}  tfidf             0   
4        128        obscene   bern  {'alpha': 10.0}  count             0   
5        130        obscene  multi   {'alpha': 0.5}  tfidf             0   
6        168         threat   bern   {'alpha': 0.5}  count             0   
7        122         threat  multi   {'alpha': 0.1}  tfidf             0   
8        128         insult   bern  {'alpha': 10.0}  count             0   
9        154         insult  multi   {'alpha': 0.1}  tfidf             0   
10       200  identity_hate   bern   {'alpha': 2.0}  count             0   
11       226  identity_hate  multi   {'alpha': 0.5}  tfidf             0   

    tokeniz